In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Locate the data
data = '../Gold Std.csv'
# Read data with pandas
dataset = pd.read_csv(data)
# Extract the values of the desired column
full_set = dataset['GOL_Returns_Rolling_Std'].values
dataset

,Unnamed: 0,Date,XAU_USD,GOL_Returns_Rolling_Std
0,0,1979-12-28,-0.678952,NaN
1,1,1980-01-01,0.000000,NaN
2,2,1980-01-02,9.326172,NaN
3,3,1980-01-03,13.309513,NaN
4,4,1980-01-04,0.000000,NaN
...,...,...,...,...
10658,10658,2021-02-10,0.280884,0.985257
10659,10659,2021-02-11,-0.915200,0.991293
10660,10660,2021-02-12,-0.104089,0.991185
10661,10661,2021-02-14,-0.128876,0.969552


In [3]:
# We split our data on the 7200 mark as it represents 70% of it
train_data = full_set[5061:9061]
test_data = full_set[9061:]
# dataset.iloc[5061]

In [4]:
# Data Normalization
from sklearn.preprocessing import MinMaxScaler
# Data must be reshaped in order to use fit_transform
training_set = train_data.reshape(-1,1)
test_set = test_data.reshape(-1,1)

sc = MinMaxScaler(feature_range=(0,1))

X_train_scaled = sc.fit_transform(training_set)

In [5]:
# Create lists for x and y train
X_train = []

y_train = []

# We will run our model on 100 timesteps
for i in range(20,4000):
    
    X_train.append(X_train_scaled[i- 20:i,0])

    y_train.append(X_train_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [6]:
# Import the necessary models from keras within tensor flow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [7]:
# Create a callback function with keras, which will monitor thr rmse of each epoch 
# And stop our model on the epoch that offers the best balance so we don't overfit our model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [8]:
# Create the layers of our model droping out and replenishing 20% of our data after each layer
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
# Include a printout of the model summary 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 20, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                2

In [9]:
model.compile(optimizer='adam',loss= 'mean_squared_error',
             metrics=[tf.keras.metrics.RootMeanSquaredError()])
# While we git our model we assign it to a variable 
history = model.fit(X_train,y_train,epochs=20,batch_size=100,callbacks=[callback], verbose = 2)

Epoch 1/20
40/40 - 15s - loss: 0.0141 - root_mean_squared_error: 0.1190
Epoch 2/20
40/40 - 3s - loss: 0.0046 - root_mean_squared_error: 0.0675
Epoch 3/20
40/40 - 3s - loss: 0.0039 - root_mean_squared_error: 0.0622
Epoch 4/20
40/40 - 3s - loss: 0.0035 - root_mean_squared_error: 0.0594
Epoch 5/20
40/40 - 3s - loss: 0.0031 - root_mean_squared_error: 0.0556
Epoch 6/20
40/40 - 3s - loss: 0.0026 - root_mean_squared_error: 0.0515
Epoch 7/20
40/40 - 3s - loss: 0.0025 - root_mean_squared_error: 0.0497
Epoch 8/20
40/40 - 4s - loss: 0.0023 - root_mean_squared_error: 0.0478
Epoch 9/20
40/40 - 3s - loss: 0.0020 - root_mean_squared_error: 0.0451
Epoch 10/20
40/40 - 3s - loss: 0.0020 - root_mean_squared_error: 0.0449
Epoch 11/20
40/40 - 3s - loss: 0.0019 - root_mean_squared_error: 0.0441
Epoch 12/20
40/40 - 3s - loss: 0.0020 - root_mean_squared_error: 0.0442
Epoch 13/20
40/40 - 3s - loss: 0.0018 - root_mean_squared_error: 0.0423
Epoch 14/20
40/40 - 3s - loss: 0.0017 - root_mean_squared_error: 0.0407


In [10]:
# Print out the number of epochs used for this model
print(len(history.history['loss']))

20


In [11]:
# Making Predictions to the test set
dataset_total = np.concatenate((train_data, test_data), axis = 0)
# Prepare our data to the test
inputs = dataset_total[len(dataset_total) - len(test_data) - 100:]
# Reshape our data to fit the upcoming for loop
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

# Create lists to which we'll append x and y values for future test
X_test = []
y_test = []

# Create a loop that will run through groups of 200 timesteps for the test sample
for i in range(20, 1411):
    X_test.append(inputs[i-20:i,0])
    y_test.append(inputs[i, 0])

In [12]:
# Turn our lists into a numpy numbers array
X_test = np.array(X_test)
y_test = np.array(y_test)

# Reshape our array into 3 dimensions so it can fit the LSTM Model
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

y_test = y_test.reshape(-1, 1)

In [13]:
 model_loss, rmse = model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Root Mean Square Error:{rmse}")

44/44 - 3s - loss: 1.9936e-04 - root_mean_squared_error: 0.0141
Normal Neural Network - Loss: 0.00019936262106057256, Root Mean Square Error:0.014119583182036877


In [14]:
predicted_returns = model.predict(X_test)
predicted_returns = sc.inverse_transform(predicted_returns)

In [17]:
predicted_returns

array([[0.74233913],
       [0.7079457 ],
       [0.67814034],
       ...,
       [0.58795494],
       [0.5896691 ],
       [0.591655  ]], dtype=float32)

In [18]:
y_test = sc.inverse_transform(y_test)


In [19]:
import plotly.express as px

In [20]:
output = pd.DataFrame({
    'predicted_returns' : [x[0] for x in predicted_returns],
    'actual_returns' : [x[0] for x in y_test]
})
output

,predicted_returns,actual_returns
0,0.742339,2.322091
1,0.707946,2.325272
2,0.678140,2.337229
3,0.658691,2.355716
4,0.651688,2.350619
...,...,...
1386,0.581736,1.992089
1387,0.584880,1.989501
1388,0.587955,1.989002
1389,0.589669,2.086871


In [24]:
px.line(output, y=["predicted_returns", "actual_returns"])

In [25]:
output.to_csv('../../Model Results/Gold-Predicted-Returns.csv')
